In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import os
import sys
import lmdb
import pickle
import numpy as np

In [ ]:
os.chdir('../') # adress to git dir
print(os.getcwd())

# Download dataset

In [ ]:
!wget https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz -O /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/mnist.npz

In [ ]:
from tensorflow.keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data(path='/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/mnist.npz')

In [ ]:
import tensorflow_datasets as tfds
ds = tfds.load('mnist', split='train', shuffle_files=True)

In [ ]:
from tensorflow import tfds

In [ ]:
# Prep folders for LMDB store

In [ ]:
from tensorflow import tfds

In [ ]:
# Prep folders for LMDB store

In [ ]:
!mkdir .data/MNIST_LMDB

In [ ]:
# !rm -rf .data/MNIST_LMDB

In [ ]:
!ls .data/MNIST_LMDB

In [ ]:
# Transform from .py calls

In [ ]:
from transformation.lmdb_transformer import LmdbTransformer

In [ ]:
validation_percentage = 30
valid_format = 'png'

In [ ]:
numpy_transformer = LmdbTransformer( validation_percentage, valid_format)

In [ ]:
numpy_transformer.transform_store_from_numpy(images =X_test,   labels=Y_test,
               lmdb_dir='.data/MNIST_LMDB', category='validation', target_size=None,
               color_mode='rgb')

In [ ]:
numpy_transformer.transform_store_from_numpy(images =X_train,  labels=Y_train,
               lmdb_dir='.data/MNIST_LMDB', category='training', target_size=None,
               color_mode='rgb')

# Read the lmdb 

#Todo
 - Env put outside of loops for write and read
 - Try the Dynamic code again with clean kernel 
 - Create Standartize as an option

In [ ]:
import os
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data')

In [ ]:
!ls -la ./MNIST_LMDB

In [ ]:
# !rm -rf /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/MNIST_LMDB/

In [ ]:
from datasets_customed.label_by_filename import read_lmdb
ds = read_lmdb('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/MNIST_LMDB/_training', 60000)

In [ ]:
from datasets_customed.label_by_filename import read_lmdb
ds_test = read_lmdb('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/MNIST_LMDB/_validation', 10000)

In [ ]:
ds_test.keys()

# Tar and unzip LMDB

In [ ]:
import os
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data')
# !ls -la

In [ ]:
!tar -cvjf mnist.tar.bz MNIST_LMDB/* 

In [ ]:
!mv mnist.tar.bz /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/MNIST_TAR

In [ ]:
!ls /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/MNIST_TAR

In [ ]:
# !split -b 50M mnist.tar.bz "mnist.tar.part"
#split -b <max size> <name of zip or dir to zip/name> <split file name beginning>

#Check if they are created
# !ls -lh mnist.tar.part*

In [ ]:
# Combine Them

#pokemon_training
# !cat mnist.tar.part* > mnist_combined.tar.bz
#!cat <split files put * at the end> > <final zip name>
# !ls -la 

!mkdir ./.test_Mnist

!tar -xf mnist.tar.bz --directory ./.test_Mnist

# !rm -rf ./.test

In [ ]:
!ls -la ./.test_Mnist/MNIST_LMDB

## LMDBImageIterator and LMDBImageGenerator

In [ ]:
os.chdir('/home/azeghost/git/Generative_Models/')

In [ ]:
os.getcwd()

In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from keras import backend as K
import logging
from utils.reporting.logging import log_message
from utils.data_and_files.file_utils import get_file_path

class LMDBImageIterator(Iterator):


    def __init__(self,
                 num_images,
                 category,
                 lmdb_dir,
                 batch_size,
                 episode_len=20,
                 episode_shift=10,
                 shuffle=True,
                 seed=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='jpeg',
                 dtype=K.floatx(),
                 ):
        
        self.category = category
        self.batch_size = batch_size
        
        
        self.lmdb_dir = lmdb_dir
        self.episode_len = episode_len
        self.episode_shift = episode_shift

        
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        print("Initializing Iterator " + category +" Number of images " +str(num_images))
        print(category,lmdb_dir, batch_size,shuffle,seed)
        self.env = lmdb.open(lmdb_dir, readonly=True)
        
        Iterator.__init__(self, num_images, batch_size, shuffle, seed)
        
        
    def __del__(self):
        self.env.close()
        
        
    def _get_batches_of_transformed_samples(self, index_array):
        print(index_array)
        images, labels = [], []
        
        if len(index_array) < self.batch_size:
            diff = self.batch_size//len(index_array) + 1
            index_array = np.repeat(index_array, diff, axis=0)[:self.batch_size]

        else:
                with self.env.begin() as txn:
                    for image_id in index_array:
                        data = txn.get(f"{image_id:08}".encode("ascii"))
                        dataset = pickle.loads(data)
                        images.append(dataset.get_image())
                        labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

                        for label in labels_list:
                            _lab = {label: eval(f'dataset.{label}')}
                            labels = {**labels, **_lab}
        return {'images': images, **labels}


In [ ]:
import glob
import hashlib
import logging
import os
import re
import warnings

from keras.preprocessing.image import ImageDataGenerator

from utils.data_and_files.data_utils import as_bytes
from utils.reporting.logging import log_message


class LMDBImageGenerator(ImageDataGenerator):
    def flow_from_lmdb_lists(self, 
                              num_images,
                              category,
                              lmdb_dir,
                              batch_size,
                              episode_len=None,
                              episode_shift=None,
                              color_mode='rgb',
                              shuffle =True,
                              seed=None
                              ):

        
          

        return LMDBImageIterator(
                             num_images = num_images,
                             category = category,
                             lmdb_dir = lmdb_dir,
                             batch_size  = batch_size,
                             episode_len = episode_len,
                             episode_shift =episode_shift,
                             shuffle = shuffle,
                             seed = seed)


def get_generators( val_lmdb_dir, val_num_images, tra_lmdb_dir, tra_num_images, 
                   batch_size, episode_len=None, episode_shift=None):

    train_datagen = LMDBImageGenerator()

    valid_datagen = LMDBImageGenerator()

    train_generator = train_datagen.flow_from_lmdb_lists(
        num_images = tra_num_images,
        category='training',
        lmdb_dir=tra_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    validation_generator = valid_datagen.flow_from_lmdb_lists(
        num_images = val_num_images,
        category='validation',
        lmdb_dir=val_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    return train_generator, validation_generator



In [ ]:
training_generator, testing_generator = get_generators(
    val_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_validation',
    val_num_images = 218,
    tra_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_training', 
    tra_num_images = 591,
    batch_size=100
)

In [ ]:
x, y = training_generator.next()

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break